In [2]:
from collections import defaultdict
import math

class OpinionAwareKnowledgeGraph:
    def __init__(self):
        self.entities = set()
        self.relations = set()
        self.graph = defaultdict(list)
        self.opinions = defaultdict(dict)

    def add_entity(self, entity):
        self.entities.add(entity)

    def add_relation(self, relation):
        self.relations.add(relation)

    def add_edge(self, subject, predicate, obj):
        self.graph[subject].append((predicate, obj))

    def initialize_opinions(self):
        for entity in self.entities:
            self.opinions[entity] = {'libOp': 0, 'conOp': 0}

    def opinion_estimation(self, training_data):
        for entity in self.entities:
            liberal_opinions = []
            conservative_opinions = []
            for sentence, ideology in training_data:
                if entity in sentence:
                    if ideology == 'liberal':
                        liberal_opinions.extend(sentence)
                    elif ideology == 'conservative':
                        conservative_opinions.extend(sentence)
            libOp = sum(lib_op == 'positive' for lib_op in liberal_opinions) - sum(lib_op == 'negative' for lib_op in liberal_opinions)
            conOp = sum(con_op == 'positive' for con_op in conservative_opinions) - sum(con_op == 'negative' for con_op in conservative_opinions)
            self.opinions[entity]['libOp'] = libOp
            self.opinions[entity]['conOp'] = conOp

    def ideology_propagation(self, background_knowledge, semantic_relations):
        extended_entities = set()
        for entity in self.entities:
            for relation, connected_entity in self.graph[entity]:
                if (entity, relation, connected_entity) in background_knowledge:
                    extended_entities.add(connected_entity)
        for entity in extended_entities:
            for relation, connected_entity in semantic_relations[entity]:
                if connected_entity in self.entities:
                    self.add_edge(entity, relation, connected_entity)
                    self.opinions[entity] = self.aggregate_opinions(semantic_relations[entity])

    def aggregate_opinions(self, relations):
        libOp_total = 0
        conOp_total = 0
        for relation in relations:
            libOp_total += math.log(relation['libOp']) if relation['libOp'] > 0 else 0
            conOp_total += math.log(relation['conOp']) if relation['conOp'] > 0 else 0
        return {'libOp': libOp_total, 'conOp': conOp_total}

    def ideology_detection(self, test_data):
        ideology_weight = 0
        for entity, opinion in self.opinions.items():
            for sentence in test_data:
                if entity in sentence:
                    opinion_orientation = sum(1 if op == 'positive' else -1 if op == 'negative' else 0 for op in sentence[entity])
                    ideology_weight += opinion['libOp'] * opinion_orientation
                    ideology_weight -= opinion['conOp'] * opinion_orientation
        ideology_weight /= len(test_data)
        if ideology_weight > 0:
            return 'liberal'
        elif ideology_weight < 0:
            return 'conservative'
        else:
            return 'neutral'  # If ideology_weight is 0, it's neutral

# Example usage:
if __name__ == "__main__":
    # Sample training and test data
    training_data = [
        ({'Criminalizing abortion': ['positive']}, 'liberal'),
        ({'Birth control': ['positive']}, 'liberal'),
        ({'Abortion harms women physically': ['negative']}, 'conservative')
    ]
    test_data = [{'Criminalizing abortion': ['positive', 'positive']}]

    # Background knowledge
    background_knowledge = {('Abortion', 'harms', 'women')}
    
    # Semantic relations
    semantic_relations = {
        'Abortion': [({'libOp': 0.6, 'conOp': 0.4}, 'harms', 'women')],
        'women': [({'libOp': 0.7, 'conOp': 0.3}, 'harmed by', 'abortion')]
    }

    # Constructing the opinion-aware knowledge graph
    graph = OpinionAwareKnowledgeGraph()
    for sentence, ideology in training_data:
        for entity in sentence.keys():
            graph.add_entity(entity)
    for subj, pred, obj in background_knowledge:
        graph.add_relation(pred)
        graph.add_edge(subj, pred, obj)
    graph.initialize_opinions()
    graph.opinion_estimation(training_data)
    graph.ideology_propagation(background_knowledge, semantic_relations)
    print("Opinion-aware knowledge graph:", graph.graph)

    # Political ideology detection
    print("Test data ideology:", graph.ideology_detection(test_data))

Opinion-aware knowledge graph: defaultdict(<class 'list'>, {'Abortion': [('harms', 'women')], 'Criminalizing abortion': [], 'Abortion harms women physically': [], 'Birth control': []})
Test data ideology: neutral
